# PIC-SURE API tutorial using CureSC database

This is a tutorial notebook, aimed for the user to be quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

<!--img src="./img/PIC-SURE_logo.png" width= "360px"> -->

Databases exposed through PIC-SURE API encompass a wide heterogeneity of architectures and data organizations underneath. PIC-SURE hide this complexity and expose the different databases in the same format, allowing researchers to focus on the analysis and medical insights, thus easing the process of reproducible sciences.

### More about PIC-SURE
PIC-SURE stands for Patient-centered Information Commons: Standardized Unification of Research Elements. The API is available in two different programming languages, python and R, allowing investigators to query databases in the same way using any of those languages.

PIC-SURE is a large project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface, allowing research scientist to get quick knowledge about variables and data available for a specific data source.

The python API is actively developed by the Avillach-Lab at Harvard Medical School.

GitHub repo:
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client



 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure you have [added your security token](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/ALS-819/Cure_Sickle_Cell#get-your-security-token). It contains explanation about how to get a security token, mandatory to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later (although earlier versions of Python 3 must work too)
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### IPython magic command

The following code loads the `autoreload` IPython extension. Although autoreload is not necessary to execute the rest of the Notebook, it does enable the notebook to reload every dependency each time python code is executed.  This will enable the notebook to take into account changes in external file imported. (e.g. user defined function stored in separate file), without having to manually reload libraries. It turns out to be very handy when developing interactively. More about [IPython Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [ ]:
%load_ext autoreload
%autoreload 2

### Installation of required python packages

Using the pip package manager, we install the packages listed in the `requirements.txt` file.

In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

Import all the external dependencies, as well as user-defined functions stored in the `python_lib` folder

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, joining_variablesDict_onCol

In [ ]:
print("NB: This Jupyter Notebook has been written using PIC-SURE API following versions:\n- PicSureClient: 0.1.0\n- PicSureHpdsLib: 1.1.0\n")
print("The PIC-SURE API libraries versions you've been downloading are: \n- PicSureClient: {0}\n- PicSureHpdsLib: {1}".format(PicSureClient.__version__, PicSureHpdsLib.__version__))

##### Set up the options for displaying tables and plots in this Notebook

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 14}

plt.rc('font', **font)

### Connecting to a PIC-SURE network

You will need the following information before connecting to the pic-sure network:
* resource_id = Id of the resource that you are trying to access. You can leave the default value for this project.
* token_file = A flat file called token.txt should contain the token retrieved from you user profile in picsure-ui.  The needs to be located at the python root folder.

In [ ]:
resource_id = "57e29a43-38c3-4c4b-84c9-dda8138badbe"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:

connection = PicSureClient.Client().connect_local(my_token)


In [ ]:
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects are created here: a `connection` and a `resource` object, using respectively the `picsure` and `hpds` libraries. 

As we will only be using one single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter** (FYI, the `connection` object is useful to get access to different databases stored in different resources). 

It is connected to the specific data source ID we specified, and enables to query and retrieve data from this source.

### Getting help with the PIC-SURE python API

Each object exposed by the PicSureHpdsLib library got a `help()` method. Calling it will print out a helper message about it. 

In [ ]:
resource.help()

For instance, this output tells us that this `resource` object got 2 methods, and it gives insights about their function. 

## Using the *variables dictionary*

Once connection to the desired resource has been established, we first need to get a quick grasp of which variables are available in the database. To this end, we will use the `dictionary` method of the `resource` object.

A `dictionary` instance offers the possibility to retrieve matching records according to a specific term, or to retrieve information about all available variables, using the `find()` method. For instance, looking for variables containing the term `Stroke` is done this way: 

Objects created by the `dictionary.find` exposes the search result using 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [ ]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("Sex")
dictionary_search.DataFrame().head()

**`.DataFrame()` enables to get the result of the dictionary search in a pandas DataFrame format** 

The dictionary provide various information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if categorical, False if continuous/numerical
- min/max: only provided for non-categorical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

Hence, it enables to:
* Use the various variables information as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variables names, to be used in the query, as shown below.


In [ ]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:3],
        "Entries": dictionary_search.entries()[0:3]})

Variable names, as currently implemented in the API, are long and got backslashes that prevent from using copy-pasting to directly select a variable name. 

However, using the dictionary to select variables can help to deal with this. Hence, one way to proceed is to retrieve the whole dictionary in the form of a pandas DataFrame, as below:

In [ ]:
plain_variablesDict = resource.dictionary().find().DataFrame()

Indeed, using the `dictionary.find()` function without arguments return every entries, as shown in the help documentation.

In [ ]:
resource.dictionary().help()

In [ ]:
plain_variablesDict.iloc[10:20,:]

### Export Full Data Dictionary to CSV

In order to export the data dictionary first we will create a Pandas dataframe called fullVariableDict

In [ ]:
fullVariableDict = resource.dictionary().find().DataFrame()

Check that the fullVariableDict dataframe contains some values.

In [ ]:
fullVariableDict.iloc[0:3,:]

In [ ]:
fullVariableDict.to_csv('data_dictionary.csv')

You should now see a data_dictionary.csv in the file explorer.

#### Variable dictionary + pandas multiIndex

Though helpful, we can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information and ease dealing with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrate how to quickly scan an select groups of related variables.

Printing the 'multiIndexed' variable Dictionary allows to quickly see the tree-like organisation of the variables. Moreover, original and simplified variable names are now stored respectively in the "name" and "simplified_name" columns.

In [ ]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [ ]:
variablesDict.loc[["CIBMTR - Cure Sickle Cell Disease"],:]

In [ ]:
# We limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in every variables pertaining to the "Transplant related" of the "CIBMTR" study.

In [ ]:
mask_study = variablesDict.index.get_level_values(0) == "CIBMTR - Cure Sickle Cell Disease"
mask_transplant = variablesDict.index.get_level_values(1) == "5 - CRF data collection track only"
transplant_variables = variablesDict.loc[mask_study & mask_transplant,:]
transplant_variables

Although pretty simple, it can be easily combined with other filters to quickly select necessary variables.

## Querying and retrieving data

Beside from the dictionary, the second cornerstone of the API is the `query` object. It is the entering point to retrieve data from the resource.

In [ ]:
my_query = resource.query()

The query object got several methods that enable to build a query

- The `query.select().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, without any record (ie subjects/rows) subsetting.

- The `query.require().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all the variables passed, and only records that do not contain any null values for those variables.

- The `query.anyof().add()` method accept variable names as string or list of strings as argument, and will allow the query to return all variables included in the list, and only records that do contain at least one non-null value for those variables.

- The `query.filter().add()` method accept variable names a variable name as strings as argument, plus additional values to filter on that given variable. The query will return this variable and only the records that do match this filter.

All those 4 methods can be combined when building a query. The record eventually returned by the query have to meet all the different specified filters.

### Building the query

Let's say we want to select a cohort from the "CIBMTR - Cure Sickle Cell Disease" using males with avascular necrosis.

In [ ]:
# Selecting all variables from "CIBMTR" study
mask_study = variablesDict.index.get_level_values(0) == "CIBMTR - Cure Sickle Cell Disease"
varnames = variablesDict.loc[mask_study, "name"]

Lets create variables that will look for Males and Avascular Necrosis.

In [ ]:
sex_var = variablesDict.loc[variablesDict["simplified_name"] == "Sex", "name"].values[0]

avascular_necrosis_varname = variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"].values[0]


In [ ]:
variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"]

Now filter by the variables by wanted values.

In [ ]:
my_query = resource.query()
my_query.select().add(avascular_necrosis_varname)
my_query.filter().add(avascular_necrosis_varname, "Yes")


In [ ]:
my_query = resource.query()
my_query.select().add(sex_var)
my_query.filter().add(sex_var, "Male")

## Retrieving the data

Once our query object is finally built, we use the `query.run` function to retrieve the data corresponding to our query

In [ ]:
query_df = my_query.getResultsDataFrame().set_index("Patient ID")

In [ ]:
query_df